In [7]:
import langchain
from langchain import PromptTemplate
import json
from langchain_openai import ChatOpenAI 

In [ ]:
HF_TOKEN = "********************"
HF_LLM_BASE_URL = "https://router.huggingface.co/fireworks-ai/inference/v1"

In [56]:
from openai import OpenAI
llm = ChatOpenAI(
    model_name="accounts/fireworks/models/deepseek-r1-0528",
    openai_api_key=HF_TOKEN,
    base_url=HF_LLM_BASE_URL,
    temperature=0.8
)

In [57]:
email_generation_template = """
You are an AI assistant designed to draft emails based on user requests.
Your output MUST be a valid JSON object.
The JSON object must contain five keys: "to", "subject", "body", "SMTP_SERVER" and "SMTP_PORT".
- "to" should be a single email address string (e.g., "recipient@example.com").
- "subject" should be a concise string for the email subject line.
- "body" should be a string containing the full email content.
- "SMTP_SERVER" should be a single SMTP_SERVER (e.g.,"smtp.gmail.com" For Gmail).
- "SMTP_PORT" should be a single value (e.g., 465  For SSL (Gmail))

Ensure the email body is professional and directly addresses the user's request.
Do NOT include any text outside the JSON object.

Context: {context}
User Request: {user_request}

JSON Output:
"""

In [58]:
EMAIL_PROMPT = PromptTemplate(
    template=email_generation_template,
    input_variables=["user_request", "context"]
)

In [ ]:
user_email_request = "Draft an email to Professor Smith(user@gmail.com) to schedule a meeting next week to discuss my Deep Learning R&D project."
email_context = "My name is KVKR. I am a student. My project ID is 1234."

In [68]:
formatted_prompt = EMAIL_PROMPT.format(
    user_request=user_email_request,
    context=email_context
)

In [69]:
print(formatted_prompt)


You are an AI assistant designed to draft emails based on user requests.
Your output MUST be a valid JSON object.
The JSON object must contain five keys: "to", "subject", "body", "SMTP_SERVER" and "SMTP_PORT".
- "to" should be a single email address string (e.g., "recipient@example.com").
- "subject" should be a concise string for the email subject line.
- "body" should be a string containing the full email content.
- "SMTP_SERVER" should be a single SMTP_SERVER (e.g.,"smtp.gmail.com" For Gmail).
- "SMTP_PORT" should be a single value (e.g., 465  For SSL (Gmail))

Ensure the email body is professional and directly addresses the user's request.
Do NOT include any text outside the JSON object.

Context: My name is KVKR. I am a student. My project ID is 1234.
User Request: Draft an email to Professor Smith(kallutlaveerakeshavareddy@gmail.com) to schedule a meeting next week to discuss my Deep Learning R&D project.

JSON Output:



In [70]:
llm_response = llm.invoke(formatted_prompt)
llm_output_str = llm_response.content

print("\n--- Raw LLM Output ---")
print(llm_output_str)

APIStatusError: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}

In [63]:
import re

In [64]:
cleaned_text = re.sub(r'<think>.*?</think>', '', llm_output_str, flags=re.DOTALL)
cleaned_text = re.sub(r'```json\s*', '', cleaned_text, flags=re.DOTALL)
cleaned_text = re.sub(r'\s*```', '', cleaned_text, flags=re.DOTALL)
cleaned_text = cleaned_text.strip()
cleaned_text = re.sub(r'\s*\n\s*', '\n', cleaned_text)
cleaned_text = re.sub(r' +', ' ', cleaned_text)

In [65]:
print(cleaned_text)

{
"to": "kvkr@office365.com",
"subject": "Meeting Request: Deep Learning R&D Project (ID 1234)",
"body": "Dear Professor Smith,\n\nI hope this message finds you well. My name is KVKR, a student working on the Deep Learning R&D project under ID 1234. I am writing to request a meeting with you sometime next week to discuss the project's progress, challenges, and next steps.\n\nCould you please share your availability? I will gladly accommodate your schedule. Thank you for your time and guidance.\n\nSincerely,\nKVKR",
"SMTP_SERVER": "smtp.office365.com",
"SMTP_PORT": 587
}


In [66]:
try:
    email_data = json.loads(cleaned_text)

    print("\n--- Parsed Email Data ---")
    print(f"To: {email_data['to']}")
    print(f"Subject: {email_data['subject']}")
    print(f"SMTP_SERVER:\n{email_data['SMTP_SERVER']}")
    print(f"SMTP_PORT:\n{email_data['SMTP_PORT']}")


except json.JSONDecodeError as e:
    print(f"\nError: Failed to parse JSON output from LLM: {e}")
    print("LLM might not have followed the JSON format correctly.")
except KeyError as e:
    print(f"\nError: Missing expected key in JSON output: {e}")


--- Parsed Email Data ---
To: kvkr@office365.com
Subject: Meeting Request: Deep Learning R&D Project (ID 1234)
SMTP_SERVER:
smtp.office365.com
SMTP_PORT:
587


In [44]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import ssl 

def send_email_via_smtp(sender_email, receiver_email, subject, body, smtp_server, smtp_port, password):
    """
    Sends an email via SMTP.
    """
    try:
        # Create a multipart message and set headers
        message = MIMEMultipart()
        message["From"] = sender_email
        message["To"] = receiver_email
        message["Subject"] = subject
        message.attach(MIMEText(body, "plain"))

        context = ssl.create_default_context() # Create a secure SSL context

        with smtplib.SMTP_SSL(smtp_server, smtp_port, context=context) as server:
            server.login(sender_email, password)
            server.send_message(message)
        print("Email sent successfully!")
        return True
    except smtplib.SMTPAuthenticationError:
        print("SMTP Authentication Error: Check your username and password or app password/less secure app access settings.")
        return False
    except Exception as e:
        print(f"An error occurred while sending email: {e}")
        return False

In [ ]:
send_email_via_smtp("sender@gmail.com", "reciever@gmail.com", email_data['subject'],
                         email_data['body'], "smtp.gmail.com", 465, "{app_password}")

Email sent successfully!


True